In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [10]:
from src.utils import loading, Spark
from src.memory_based import Memory_based_CF
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from tqdm.notebook import tqdm
from sklearn import metrics

In [4]:
from scipy import sparse
import numpy as np
import pandas as pd

In [5]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [6]:
datas = loading(spark, '../data/interim')

In [7]:
cast_int = lambda df: df.select([F.col(c).cast('int') for c in ['userId', 'movieId']] + \
                                 [F.col('rating').cast('float')])
training, test = cast_int(datas['train_0.75_0.25']), cast_int(datas['test_0.75_0.25'])

In [8]:
cf = Memory_based_CF(spark, 'user')
cf.fit(training)
pred = cf.predict(test).toPandas()

In [11]:
metrics.mean_squared_error(pred.rating, pred.prediction)

1.3436497540830965

In [12]:
cf = Memory_based_CF(spark, 'item')
cf.fit(training)

In [13]:
pred = cf.predict(test).toPandas()

100%|██████████| 28961/28961 [00:18<00:00, 1599.59it/s]


In [14]:
metrics.mean_squared_error(pred.rating, pred.prediction)

1.353190988204929

In [15]:
pred

,userId,movieId,rating,prediction
0,117490,1127,3,3.253030
1,117490,2109,3,3.273222
2,117490,1095,4,3.633355
3,117490,3893,3,3.373658
4,137652,1032,3,3.611988
...,...,...,...,...
28956,99406,88744,3,3.638721
28957,9954,8798,4,4.313750
28958,99780,8665,4,2.142125
28959,99893,69757,5,3.353411


In [18]:
test.toPandas()

,userId,movieId,rating
0,100,1608,3.5
1,100031,2288,4.0
2,100031,58559,3.5
3,100060,5903,2.5
4,100060,6936,3.5
...,...,...,...
30337,99927,8528,3.0
30338,99945,2707,3.5
30339,99959,356,3.5
30340,99990,5015,0.5
